In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

/Users/arvind/miniconda3/envs/treehacks24/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder = SentenceTransformer('all-mpnet-base-v2')

In [3]:
classes = ["clarify", "email", "link", "schedule", "todos", "unknown"]

In [4]:
# Walk through all files in a directory
import os
lines = []
for class_ in classes:
    # Read contents of file
    with open('data/action_classification/' + class_ + '.txt', 'r') as f:
        lines += f.readlines()

inputs = encoder.encode(lines)
inputs = torch.Tensor(inputs)
print(inputs.shape)

torch.Size([600, 768])


In [5]:
outputs = []
for class_ in classes:
    output_1he = torch.Tensor([0] * len(classes))
    output_1he[classes.index(class_)] = 1
    for i in range(100):
        outputs.append(output_1he)

outputs = torch.stack(outputs)
print(outputs.shape)

torch.Size([600, 6])


In [6]:
# Split data
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [7]:
class ActionClassifier(torch.nn.Module):
    def __init__(self):
        super(ActionClassifier, self).__init__()
        self.fc1 = nn.Linear(768, 384)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(384, 384)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(384, 384)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(384, 384)
        self.dropout4 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(384, len(classes))
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = self.dropout1(x)
        x = F.gelu(self.fc2(x))
        x = self.dropout2(x)
        x = F.gelu(self.fc3(x))
        x = self.dropout3(x)
        x = F.gelu(self.fc4(x))
        x = self.dropout4(x)
        x = F.gelu(self.fc5(x))
        x = self.softmax(x)
        return x

model = ActionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [8]:
best_test_loss = float('inf')
for epoch in range(5000):
    optimizer.zero_grad()
    preds_train = model(inputs_train)
    train_loss = criterion(preds_train, outputs_train)
    train_loss.backward()
    train_acc = (preds_train.argmax(1) == outputs_train.argmax(1)).float().mean()
    optimizer.step()

    preds_test = model(inputs_test)
    test_loss = criterion(preds_test, outputs_test)
    test_acc = (preds_test.argmax(1) == outputs_test.argmax(1)).float().mean()
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'action_classifier.pt')

    print('Epoch {} train loss: {} train acc: {} test loss: {} best test loss: {} test acc: {}'.format(epoch, train_loss, train_acc, test_loss, best_test_loss, test_acc))

Epoch 0 train loss: 1.7918013334274292 train acc: 0.17499999701976776 test loss: 1.791577935218811 best test loss: 1.791577935218811 test acc: 0.20000000298023224
Epoch 1 train loss: 1.7918206453323364 train acc: 0.15208333730697632 test loss: 1.7916284799575806 best test loss: 1.791577935218811 test acc: 0.1666666716337204
Epoch 2 train loss: 1.7918038368225098 train acc: 0.15416666865348816 test loss: 1.7915575504302979 best test loss: 1.7915575504302979 test acc: 0.20000000298023224
Epoch 3 train loss: 1.7918046712875366 train acc: 0.15833333134651184 test loss: 1.7916004657745361 best test loss: 1.7915575504302979 test acc: 0.18333333730697632
Epoch 4 train loss: 1.791772723197937 train acc: 0.18958333134651184 test loss: 1.7915751934051514 best test loss: 1.7915575504302979 test acc: 0.2083333283662796
Epoch 5 train loss: 1.7917861938476562 train acc: 0.17499999701976776 test loss: 1.7916170358657837 best test loss: 1.7915575504302979 test acc: 0.15000000596046448
Epoch 6 train lo

In [9]:
action_classifier = ActionClassifier()
action_classifier.load_state_dict(torch.load('action_classifier.pt'))
action_classifier.eval()

ActionClassifier(
  (fc1): Linear(in_features=768, out_features=384, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=384, out_features=384, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=384, out_features=384, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (fc4): Linear(in_features=384, out_features=384, bias=True)
  (dropout4): Dropout(p=0.2, inplace=False)
  (fc5): Linear(in_features=384, out_features=6, bias=True)
  (softmax): Softmax(dim=1)
)

In [12]:
def infer(text):
    input_ = torch.Tensor(encoder.encode([text]))
    output = action_classifier(input_)
    return classes[output.argmax(1)]